In [1]:
import numpy as np
import sys
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.preprocessing.text import one_hot
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
text= open("lyrics_kanye.txt").read()
chars = sorted(list(set(text.split(" ")))) #split into a sorted list of characters
text_list = text.split(" ")
text_list_size = len(text_list)
vocab_size = len(chars)
char_size = len(text) 
print(vocab_size)
print(char_size)

13944
1446286


In [3]:
ix_to_char = {ix:char for ix, char in enumerate(chars)} #create a dictionary of the spot of each character
char_to_ix = {char:ix for ix, char in enumerate(chars)}

In [4]:
#create number of sequences
sequence_cap = 3 #want to make sure it doesn't overuse unique patterns
#below to "End pattern organization from: https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/"
dataX = []
dataY = []
for i in range(0, text_list_size - sequence_cap, sequence_cap):
	seq_in = text_list[i:i + sequence_cap]
	seq_out = text_list[i + sequence_cap]
	dataX.append([char_to_ix[char] for char in seq_in])
	dataY.append(char_to_ix[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X =  np.reshape(dataX, (n_patterns, sequence_cap,1))
# normalize
X = X / float(vocab_size)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
#End pattern organization.

Total Patterns:  97223


In [5]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [6]:
#from machinelearning site above.....
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
97223/97223 [==============================] - 11s 110us/step - loss: 6.9200

Epoch 00001: loss improved from inf to 6.92004, saving model to weights-improvement-01-6.9200.hdf5
Epoch 2/20
97223/97223 [==============================] - 9s 94us/step - loss: 6.6702

Epoch 00002: loss improved from 6.92004 to 6.67016, saving model to weights-improvement-02-6.6702.hdf5
Epoch 3/20
97223/97223 [==============================] - 9s 89us/step - loss: 6.6542

Epoch 00003: loss improved from 6.67016 to 6.65418, saving model to weights-improvement-03-6.6542.hdf5
Epoch 4/20
97223/97223 [==============================] - 9s 96us/step - loss: 6.6472

Epoch 00004: loss improved from 6.65418 to 6.64717, saving model to weights-improvement-04-6.6472.hdf5
Epoch 5/20
97223/97223 [==============================] - 9s 90us/step - loss: 6.6371

Epoch 00005: loss improved from 6.64717 to 6.63705, saving model to weights-improvement-05-6.6371.hdf5
Epoch 6/20
97223/97223 [============================

In [68]:
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", '---'.join([ix_to_char[value] for value in pattern]), "\"")

Seed:
" problems---of---my "


In [102]:
# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(vocab_size)
    prediction = model.predict(x, verbose=0)
    p_better=[prediction[0]]
    p_bettah=[p_better[0]]
    #index = np.argmax(prediction)#maybe not most likely 
    index = np.random.choice(chars, size=sequence_cap, p=prediction[0]) #this is now the list of next 3 words
    #index = np.random.choice(prediction, size=None, replace=True, p=None)
    #result = ix_to_char[index]
    #seq_in = [ix_to_char[value] for value in pattern]
    #print(index)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")
print(vocab_size) 
print(len(p_bettah))
print(prediction[0])

ValueError: a and p must have same size

In [94]:
import heapq
import operator
top_ten = zip(*heapq.nlargest(10, enumerate(prediction), key=operator.itemgetter(1)))[0]
print(top_ten)

TypeError: 'zip' object is not subscriptable

In [84]:
import numpy as np
import random as pr

def rargmax(vector):
    """ Argmax that chooses randomly among eligible maximum indices. """
    m = np.amax(vector)
    indices = np.nonzero(vector == m)[0]
    return pr.choice(indices)

if __name__ == '__main__':
    test = [0,1,2,2]
    for i in range(10):
        print(rargmax(test))

3
3
2
2
3
2
2
2
3
3


In [70]:
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(vocab_size)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = ix_to_char[index]
	seq_in = [ix_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

iitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiitheiithe